In [1]:
!pip install schedule

In [10]:
import google.generativeai as genai
from google.colab import userdata
import json
import os
from datetime import datetime, timedelta
import schedule
import time
import threading
import sys

# API 키 설정
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

# 저장 파일 경로 설정
# 일정과 대화 내용 저장.
SCHEDULE_PATH = os.path.join(os.getcwd(), "schedule.json")
CHAT_HISTORY_PATH = os.path.join(os.getcwd(), "chat_history.json")

# 대화 내용 저장 함수
def save_to_file(data, filename):
    try:
        with open(filename, "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        print(f"데이터가 '{filename}' 파일에 저장되었습니다.")
    except Exception as e:
        print(f"데이터 저장에 실패하였습니다: {e}")

# 이전대화 불러오기 함수
def load_from_file(filename):
    try:
        if os.path.exists(filename):
            with open(filename, "r", encoding="utf-8") as file:
                return json.load(file)
        else:
            return []
    except Exception as e:
        print(f"데이터 불러오기에 실패하였습니다: {e}")
        return []

# 전역 변수로 대화 상태 저장
global_chat_history = []
global_schedule_list = []

# 대화 기반 일정 처리 함수
def process_schedule_with_ai(chat, schedule_list, chat_history, user_input):

    schedule_info = "\n".join([
        f"{idx+1}. {item['datetime']} - {item['title']}" for idx, item in enumerate(schedule_list)
    ]) if schedule_list else "현재 저장된 일정이 없습니다."

    system_prompt = f"""
    당신은 일정 관리 AI 비서입니다.
    일정 관리 명령어:
    - '일정 추가': 새로운 일정을 생성합니다.
    - '일정 확인': 저장된 일정 목록을 보여줍니다.
    - '일정 삭제': 일정을 삭제할 수 있습니다.

    현재 저장된 일정:
    {schedule_info}
    """

    # 시스템 프롬프트 전송
    chat.send_message(system_prompt)

    # 현재 타임스탬프 추가
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # AI에게 요청을 처리하도록 요청
    response = chat.send_message(user_input)
    print(f"AI: {response.text}")

    # 대화 내용 저장
    chat_history.append({
        "role": "user",
        "parts": [{"text": user_input}],
        "timestamp": current_time
    })
    chat_history.append({
        "role": "model",
        "parts": [{"text": response.text}],
        "timestamp": current_time
    })

    # 전역 변수 업데이트
    global global_chat_history, global_schedule_list
    global_chat_history = chat_history
    global_schedule_list = schedule_list

    # 일정 관리 명령어 처리
    if "일정 추가" in user_input:
        title = input("일정 제목: ")
        date = input("날짜 (YYYY-MM-DD): ")
        time = input("시간 (HH:MM): ")
        print(add_schedule(schedule_list, title, date, time))
    elif "일정 삭제" in user_input:
        print(delete_schedule(schedule_list))

# 종료 함수
def save_and_exit(schedule_list, chat_history):
    print("\n대화 내용을 저장하고 프로그램을 종료합니다.")
    save_to_file(chat_history, CHAT_HISTORY_PATH)
    save_to_file(schedule_list, SCHEDULE_PATH)

# AI 일정 관리 비서 프로그램
def ai_schedule_manager():
    print("안녕하세요. AI 일정 관리 비서입니다.")

    # 이전 대화 및 일정 불러오기
    schedule_list = load_from_file(SCHEDULE_PATH)
    chat_history = load_from_file(CHAT_HISTORY_PATH)

    # AI 모델 초기화
    model = genai.GenerativeModel("gemini-1.5-flash")

    # 최근 5개의 대화만 선택
    formatted_history = []
    for entry in chat_history[-10:]:  # 사용자와 모델 메시지 각각 5개씩
        if entry.get('role') == 'user':
            formatted_history.append({
                'role': 'user',
                'parts': [{'text': entry['parts'][0]['text']}]
            })
        elif entry.get('role') == 'model':
            formatted_history.append({
                'role': 'model',
                'parts': [{'text': entry['parts'][0]['text']}]
            })

    # 대화 시작
    chat = model.start_chat(history=formatted_history)

    # 스케줄러 실행
    scheduler_thread = threading.Thread(target=run_scheduler, daemon=True)
    scheduler_thread.start()

    while True:
        user_input = input("You: ")
        if user_input.lower() == "종료":
            save_and_exit(schedule_list, chat_history)
            break
        elif "현재시간" in user_input:
            print(get_current_time())
        else:
            process_schedule_with_ai(chat, schedule_list, chat_history, user_input)

# 현재 시간 확인 함수
def get_current_time():
    now = datetime.now()
    return f"현재 시간은 {now.strftime('%Y-%m-%d %H:%M:%S')}입니다."

# 알림 함수
def notify(title, datetime_str):
    print(f"[알림] '{title}' 일정이 시작됩니다! ({datetime_str})")

# 일정 추가 함수
def add_schedule(schedule_list, title, date, time):
    datetime_str = f"{date} {time}"
    try:
        datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M")
        schedule_list.append({"title": title, "datetime": datetime_str})
        save_to_file(schedule_list, SCHEDULE_PATH)

        # 스케줄 알림 설정
        schedule_time = datetime_obj.strftime("%H:%M")
        schedule.every().day.at(schedule_time).do(notify, title, datetime_str)

        return f"'{title}' 일정이 추가되었습니다: {datetime_str}"
    except ValueError:
        return "날짜나 시간 형식이 올바르지 않습니다. (예: 2025-01-25 15:00)"

# 일정 조회 함수
def view_schedule(schedule_list):
    if not schedule_list:
        return "현재 저장된 일정이 없습니다."
    return "\n".join([
        f"{idx+1}. {item['datetime']} - {item['title']}" for idx, item in enumerate(schedule_list)
    ])

# 일정 삭제 함수
def delete_schedule(schedule_list):
    if not schedule_list:
        return "삭제할 일정이 없습니다."
    print("삭제 가능한 일정 목록:")
    for idx, item in enumerate(schedule_list):
        print(f"{idx + 1}. {item['datetime']} - {item['title']}")
    try:
        choice = int(input("삭제할 일정 번호를 입력하세요: ")) - 1
        if 0 <= choice < len(schedule_list):
            removed_item = schedule_list.pop(choice)
            save_to_file(schedule_list, SCHEDULE_PATH)
            return f"'{removed_item['title']}' 일정이 삭제되었습니다."
        else:
            return "잘못된 번호입니다."
    except ValueError:
        return "유효한 숫자를 입력하세요."
# 스케줄러 실행 함수
def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(1)



# 프로그램 실행
if __name__ == "__main__":
    ai_schedule_manager()

안녕하세요. AI 일정 관리 비서입니다.
You: 일정 확인
AI: 현재 저장된 일정은 다음과 같습니다:

1. 2000-03-28 13:11 - 테스트

You: 일정 추가
AI: 어떤 일정을 추가하시겠습니까?  날짜, 시간, 그리고 내용을 알려주세요.  예시:  2024년 10월 27일 오후 3시 - 미팅



일정 제목: 테스트2
날짜 (YYYY-MM-DD): 2000-01-01
시간 (HH:MM): 11:11
데이터가 '/content/schedule.json' 파일에 저장되었습니다.
'테스트2' 일정이 추가되었습니다: 2000-01-01 11:11
You: 일정 삭제
AI: 어떤 일정을 삭제하시겠습니까?  번호를 입력해주세요. (예: 1)

삭제 가능한 일정 목록:
1. 2000-03-28 13:11 - 테스트
2. 2000-01-01 11:11 - 테스트2
삭제할 일정 번호를 입력하세요: 2
데이터가 '/content/schedule.json' 파일에 저장되었습니다.
'테스트2' 일정이 삭제되었습니다.
You: 일정 확인
AI: 현재 저장된 일정은 다음과 같습니다:

1. 2020-03-28 13:11 - 테스트

You: 종료

대화 내용을 저장하고 프로그램을 종료합니다.
데이터가 '/content/chat_history.json' 파일에 저장되었습니다.
데이터가 '/content/schedule.json' 파일에 저장되었습니다.
